In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
import numpy as np

import deep_fib
import utils

In [10]:
train, test = utils.data.get_train_test_split(0.1, 42)
dataset = utils.data.Marconi100Dataset(train, normalize=True)

100%|██████████| 224/224 [03:31<00:00,  1.06it/s]


TODO: salvare il dataset processato come nuovo dataset:
```
dataset
    |->unfolded
    |   |->r205n12_0.gzip
    |   |->r205n12_1.gzip
    |   |->...
    |   |->r256n18_n.gzip
    |->masks
    |   |->r205n12_mask_0.gzip
    |   |->r205n12_mask_1.gzip
    |   |->...
    |   |->r256n18_mask_n.gzip
    |->labels
        |->r205n12_label_0.gzip
        |->r205n12_label_1.gzip
        |->...
        |->r256n18_label_n.gzip
```

In [ ]:
dataset_deep_fib = deep_fib.data.DeepFIBDataset(dataset, 1024, 3)

In [ ]:
original, mask, labels = dataset_deep_fib[1]

utils.nn.plot_dataframe(pd.DataFrame(original.numpy()), pd.Series(labels.values))

In [5]:
original.shape

torch.Size([13383, 460])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time_len=1024
nel=3 
input_dim=original.shape[1]

model = deep_fib.sci_net.SCINet(
    output_len=time_len,
    input_len=time_len,
    num_encoder_levels=nel,
    hidden_decoder_sizes=[2048],
    block_config=deep_fib.sci_net.SCIBlockCfg(
        input_dim=input_dim,
        hidden_size=16,
        kernel_size=3,
        dropout=0.5,
    )
).float().to(device)

x = original[:time_len].unsqueeze(0).float().to(device)
y = model(x)
print(y.shape)


Initializing Scinet...
Initializing Encoder Tree...
Initializing Decoder...
